### 나이브 베이즈 분류기 - 트윗.

**<span style="color:blue">목표</span>**: 베이즈 정리를 적용하여 자연어 분류 모형을 만들어 본다.

<br>
1. 베이즈 정리. <br>

 $P(A| W_1, W_2, W_3,...) = {P(W_1, W_2, W_3,...|A) \cdot P(A) \over P(W_1, W_2, W_3,....)}$ <br>
 $P(B| W_1, W_2, W_3,...) = {P(W_1, W_2, W_3,...|B) \cdot P(B) \over P(W_1, W_2, W_3,....)}$

<br>
2. 동일한 분모를 무시하면 다음 비례관계가 성립된다. <br>

 $P(A| W_1, W_2, W_3, \ldots) \sim P(W_1, W_2, W_3,...|A) \cdot P(A) $ <br>
 $P(B| W_1, W_2, W_3, \ldots) \sim P(W_1, W_2, W_3,...|B) \cdot P(B) $
 
<br>
3. 독립적인 단어 분포를 전제하면 다음과 같이 분해할 수 있다. <br>

 $P(A| W_1, W_2, W_3, \ldots) \sim  P(W_1|A)\cdot P(W_2|A)\cdot P(W_3|A) \cdots P(A) $ <br>
 $P(B| W_1, W_2, W_3, \ldots) \sim P(W_1|B)\cdot P(W_2|B)\cdot P(W_3|B) \cdots P(B) $

<br>
4. 이제는 로그함수를 적용해 본다. <br>

$Log_A \sim  Log(P(W_1|A)) +  Log(P(W_2|A)) + Log(P(W_3|A)) + \ldots + Log(P(A)) $ <br>
$Log_B \sim  Log(P(W_1|B)) +  Log(P(W_2|B)) + Log(P(W_3|B)) + \ldots + Log(P(B)) $
<br>

**<span style="color:blue">결론</span>**:  $Log_A$와 $Log_B$를 비교해서 큰 쪽으로 인식!

In [ ]:
import pandas as pd
import numpy as np
import re

#### 1. 학습 자료를 읽어와서 전처리한다:

In [ ]:
# A 유형의 트윗을 읽어온다. 
f = open("../data/tweets_A.txt","r")
ta = f.readlines()
f.close()

# B 유형의 트윗을 읽어온다. 
f = open("../data/tweets_B.txt","r")
tb = f.readlines()
f.close()

In [ ]:
print(f"A 유형의 수 : {len(ta)}")
print(f"B 유형의 수 : {len(tb)}")

In [ ]:
# 전처리 해서 단어별 도수표를 만들어주는 함수.
def preprocessor(tweets):
    freq_dict = {}
    for a_line in tweets:
        a_line = a_line.lower()               # 소문자화.
        a_line = re.sub(r"\W"," ",a_line)     # 특수문자 제거.
        a_line = re.sub(r"\d", " ", a_line)   # 숫자 제거.
        a_line = re.sub("a|the|and|or|because|at", " ",a_line)  #  불용어 제거.
        a_line = a_line.split()               # 분절.
        for a_word in a_line:
            if len(a_word) > 3:               # 길이가 최소 조건을 충족하는 단어만 사용.
                if a_word in freq_dict:
                    freq_dict[a_word] += 1    # 카운트 누적.
                else:
                    freq_dict[a_word] = 2     # 처음 발견. 기본값 1 + 누적 1.
    return freq_dict

#### 2. 학습 모형을 준비해 둔다:

In [ ]:
# Series로 변환.
freq_a = pd.Series(preprocessor(ta)).sort_values(ascending=False)
freq_b = pd.Series(preprocessor(tb)).sort_values(ascending=False) 

In [ ]:
# Vocabulary 사전 크기 확인.
print("Size of A: {}".format( len(freq_a)))
print("Size of B: {}".format( len(freq_b)))

In [ ]:
# Vocabulary 수 맞춤.
n_voca = 300
freq_a = freq_a.iloc[:n_voca]
freq_b = freq_b.iloc[:n_voca]

In [ ]:
# 로그 확률로 변환.
freq_a_sum = freq_a.sum()
freq_b_sum = freq_b.sum()
log_prob_a = dict(np.log(freq_a/freq_a_sum))    # Log(P(W|A))
log_prob_b = dict(np.log(freq_b/freq_b_sum))    # Log(P(W|B))

#### 3. 테스트 데이터를 읽어온다:

In [ ]:
# 테스트 트윗을 읽어온다 (X_test). 
f = open("../data/tweets_test.txt","r")
tt = f.readlines()
f.close()

# 테스트 트윗의 유형 정보를 읽어온다 (Y_test).
f = open("../data/tweets_test_class.txt","r")
Y_test_raw = f.read()                 # 한 덩어리로 읽어온다.
Y_test = Y_test_raw.split()           # 분절을 통해서 깔끔히!
f.close()

#### 4. 예측을 실시한다:

In [ ]:
Y_pred = []
for a_sentence in tt:
    log_prob_sum_a = 0.
    log_prob_sum_b = 0.
    a_sent_preprocessed = preprocessor([a_sentence])
    for a_word, a_freq in a_sent_preprocessed.items():
        if a_word in log_prob_a:
            log_prob_sum_a += log_prob_a[a_word]*a_freq
        else:
            log_prob_sum_a += np.log(1.0/freq_a_sum)*a_freq
            
        if a_word in log_prob_b:
            log_prob_sum_b += log_prob_b[a_word]*a_freq
        else:
            log_prob_sum_b += np.log(1.0/freq_b_sum)*a_freq
            
    if (log_prob_sum_a > log_prob_sum_b):
        Y_pred.append("A")
    else:
        Y_pred.append("B")

In [ ]:
# 정확도 계산.
correct = pd.Series([ x == y for (x,y) in zip(Y_pred, Y_test) ])
print("Accuracy : {}".format(correct.mean()))